### Importing Necessary libraries

In [1]:
import numpy as np
from scipy.optimize import linprog
import imageio

### Loading the data
Provide path to the 'C.npy', 'y.npy' and 'A_inv.npy' files

In [2]:
c_path = "C.npy" #path to the C.npy file
y_path = "y.npy" #path to the y.npy file
A_inv_path = "A_inv.npy" #path to the A_inv.py file

### The function that solves the given optimization problem
$$
\min _{s}\|s\|_{1} \\
\text{such that } \|y-C s\|_{2}^{2}=0 $$

We solve the given problem using an equivalent linear program. To do so we define:

$$ s = u - v, \; {u}_{i} = \max \left\{ {s}_{i}, 0 \right\}, \; {v}_{i} = \max \left\{ -{s}_{i}, 0 \right\} $$

and in place of $\|y-C s\|_{2}^{2}=0$ we take $y = C s$

Then the problem becomes:

\begin{align*}
\arg \min_{u, v} \: & \: \sum_{i = 1}^{n} {u}_{i} + {v}_{i} \\
\text{subject to} \: & \: C \left( u - v \right) = y \\
& \: u \succeq \boldsymbol{0} \\
& \: v \succeq \boldsymbol{0}
\end{align*}

In [3]:
def solve(A, b):
  numRows = A.shape[0] 
  numCols = A.shape[1]

  print("Number of Rows of A = " + str(numRows))
  print("Number of Columns of A = " + str(numCols))

  vF = np.ones([2*numCols, 1])

  mAeq = np.concatenate((A, -A), axis=1)
  vBeq = b

  vLowerBound = np.full([2 * numCols, 1], 0)
  vUpperBound = np.full([2 * numCols, 1], np.inf)
  Bounds = np.concatenate((vLowerBound, vUpperBound), axis=1)

  result = linprog(vF, A_eq=mAeq, b_eq=vBeq, bounds=Bounds)
  vUV = result.x
  s = vUV[0:numCols] - vUV[numCols:];

  return s

The below code finds the sparse vector 's' using the above function

In [4]:
A = np.load(c_path)
b = np.load(y_path)
s = solve(A, b)
print("Done")
print(s)
#np.save('s', s) #if you need to save the sparse vector s, uncomment this line

Number of Rows of A = 2760
Number of Columns of A = 9200
Done
[ 1.30381445e+04  3.88050523e+02  2.53619249e+02 ...  8.15972781e-09
  1.65101116e-07 -1.76997052e-07]


The below code finds the vector 'x' corresponding to the original image using the matrix 'A'

In [5]:
A_inv = np.load(A_inv_path)
A_inv = A_inv.astype('float64')
A = np.linalg.inv(A_inv) 
x = A.dot(s)
print("Done")
print(x)
#np.save('x', x) #if you need to save the vector x, uncomment this line

Done
[138.15180457 193.55758403 204.3124893  ...   2.02011058  -1.59989573
   1.39281639]


The below code converts the vector 'x' back to image using 'imageio' library

*make sure to change the dimensions here properly.* \
*For example if the image has width 92 px and height 100 px, the (100,92) goes in the bracket*

In [6]:
#make sure to change the dimensions here properly. 
#For example if the image has width 92 px and height 100 px, the (100,92) goes in the bracket
y = x.reshape((100,92), order='F')
imageio.imwrite('uncorrupted_image.jpg', y)

Lossy conversion from float64 to uint8. Range [-57.83561344066531, 297.599168495918]. Convert image to uint8 prior to saving to suppress this warning.
